# Retirement Planning and Simulation

### BUSI 721: Data-Driven Finance I
### Kerry Back, Rice University

### Review

$$\text{PV} = \frac{\text{FV}}{(1+r)^n} \quad \Leftrightarrow \quad \text{FV} = \text{PV} \times (1+r)^n$$

$$B_0 = P\left[\frac{1}{1+r} + \frac{1}{(1+r)^2} + \cdots +  \frac{1}{(1+r)^n}\right]$$




### Retirement Planning

- Hope to spend $y$ dollars per year during $n$ years of retirement
- Save $x$ dollars per year during $m$ years of working
- Expect to earn $r$ per year on investments.
- How large does $x$ need to be?
- Timeline: 
  - years $ 1, \ldots, m \rightarrow$ save $x$
  - years $m+1, \ldots, m+n \rightarrow$ spend $y$.

- Compute PV of spending at year $m$ (standard annuity formula)
- Discount to present - divide by $(1+r)^m$
- Match PV of savings:
$$x \times \left[\frac{1}{1+r} + \cdots +\frac{1}{(1+r)^m}\right]$$
$$ = y\times\frac{1}{(1+r)^m}\left[\frac{1}{1+r} + \cdots +\frac{1}{(1+r)^n}\right]$$

- Or match the values at the end of year m: account balance = retirement target
- Multiply both PVs by $(1+r)^m$ to get these FVs:

$$x \times \left[(1+r)^{m-1} + \cdots +1\right]$$
$$ = y\times \left[\frac{1}{1+r} + \cdots +\frac{1}{(1+r)^n}\right]$$

### Example

In [59]:
import numpy_financial as npf

spending = 100000
num_spending_years = 25
num_saving_years = 30
r = 0.06

pv_spending_at_retirement = npf.pv(
    rate=r, 
    nper=num_spending_years, 
    pmt=-spending
)

print(f"We need to have ${pv_spending_at_retirement:,.0f} at retirement.")


We need to have $1,278,336 at retirement.


# MATCH PRESENT VALUES

In [60]:
savings = - npf.pmt(
    pv=pv_spending_at_retirement / (1+rate)**num_saving_years, 
    rate=r, 
    fv=0, 
    nper=num_saving_years
)

print(f"We need to save ${savings:,.0f} each year.")

We need to save $16,170 each year.


# MATCH FUTURE VALUES

In [61]:
savings = - npf.pmt(
    pv=0, 
    rate=r, 
    fv=pv_spending_at_retirement, 
    nper=num_saving_years
)

print(f"We need to save ${savings:,.0f} each year.")

We need to save $16,170 each year.


### Growing Savings

- Save $x$ first year, $x(1+g)$ second year, $x(1+g)^2$ third year, etc.
- E.g., $x=20,000$, $g=0.05$, second year is $21,000$, third year is $22,050$, etc.
- FV of savings:
$$x(1+r)^{m-1} + x(1+g)(1+r)^{m-2} + \cdots + x(1+g)^{m-1}$$
$$=x\bigg[(1+r)^{m-1} + (1+g)(1+r)^{m-2} + \cdots + (1+g)^{m-1} \bigg]$$

### Match FVs

In [62]:
import numpy as np 

g = 0.03
m = num_saving_years

factors = (1+g) ** np.arange(m) 
factors *= (1+r) ** np.arange(m-1, -1, -1)
x = pv_spending_at_retirement / np.sum(factors)

print(f"We need to save ${x:,.0f} each year.")

We need to save $11,564 each year.


### Inflation and Real Rate of Return

- Inflation rate is % change in Consumer Price Index (CPI)
- Real rate of return is inflation-adjusted rate
- Example:
  - Item costs $\$100$ today
  - Can earn $8\%$ on investments
  - Inflation is $3\%$
  - Instead of buying today, you could invest $\$100$.
  - Have $\$108$ in one year, item costs $\$103$, buy $1$ and have $\$5$ left over
  - Extra $\$5$ will buy $5/103$ units.  Real rate of return is $5/103$.

- $5/103 = (108-103)/103 = 108/103 - 1 = 1.08/1.03 - 1$
- Real rate of return is
$$r_{\text{real}} = \frac{1+r_{\text{nominal}}}{1+\text{inflation}} - 1$$

### Retirement Planning and Inflation

- Do everything with expected real rate of return
- If savings are expected to grow, use the real growth rate
$$ g_{\text{real}} = \frac{1+g_{\text{nominal}}}{1+\text{inflation}} - 1$$
- Then retirement spending will be in today's dollars.
- [https://learn-investments.rice-business.org/borrowing-saving/inflation](https://learn-investments.rice-business.org/borrowing-saving/inflation)

### Simulation

- Annual returns are approximately normally distributed.
- And are approximately independent from one year to the next.
- Simulate random normals with np.random.normal.

In [63]:
mean = 0.1
stdev = 0.15
np.random.seed(0)
np.random.normal(loc=mean, scale=stdev)

0.36460785189514955

### Simulate Returns for $n$ Years

In [64]:
n = 10
np.random.seed(0)

rets = np.random.normal(
    loc=mean,
    scale=stdev,
    size=n
)
rets

array([ 0.36460785,  0.16002358,  0.2468107 ,  0.43613398,  0.3801337 ,
       -0.04659168,  0.24251326,  0.07729642,  0.08451717,  0.16158978])

### Compound Returns for $n$ Years

- How much would $1 grow to?
- What is the total return over the $n$ years?

In [65]:
print(f"$1 would grow to ${np.prod(1+rets): .3f}")
print(f"the total return is {np.prod(1+rets)-1: .1%}")

$1 would grow to $ 6.289
the total return is  528.9%


### Repeat $s$ Times

In [66]:
s = 5
np.random.seed(0)

rets = np.random.normal(
    loc=mean,
    scale=stdev,
    size=(n, s)
)
np.prod((1+rets), axis=0)

array([1.30568504, 4.01010011, 2.92173927, 2.62512839, 4.17660966])

### Distribution of the Compound Return

- Compounding produces positive skewness
- So, the median is below the mean
- The skewness is greater and the difference between median and mean is larger when there is more risk.

In [67]:
s = 1000
np.random.seed(0)

rets = np.random.normal(
    loc=mean,
    scale=stdev,
    size=(n, s)
)
compound_rets = np.prod((1+rets), axis=0) - 1

import pandas as pd 
pd.Series(compound_rets).describe()

count    1000.000000
mean        1.525291
std         1.123597
min        -0.464922
25%         0.737312
50%         1.330335
75%         2.061266
max         7.906556
dtype: float64

[https://learn-investments.rice-business.org/risk/long-run](https://learn-investments.rice-business.org/risk/long-run)